In [1]:
import seaborn as sns
df=sns.load_dataset('tips')
df.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [4]:
df['sex'].unique()

array([0, 1])

In [5]:
from sklearn.preprocessing import LabelEncoder
encoder=LabelEncoder()
df['sex']=encoder.fit_transform(df['sex'])

In [6]:
df['sex'].value_counts()

1    157
0     87
Name: sex, dtype: int64

In [7]:
X=df.drop(labels=['sex'],axis=1)
y=df['sex']

In [8]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.20,random_state=42)

In [9]:
from sklearn.impute import SimpleImputer 
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [15]:
categorical_cols = ['time', 'smoker','day']
numerical_cols = ['total_bill', 'tip','size']

In [16]:
num_pipeline=Pipeline(
    steps=[
        ('imputer',SimpleImputer(strategy='median')), 
        ('scaler',StandardScaler())
    ]

)
cat_pipeline=Pipeline(
                steps=[
                ('imputer',SimpleImputer(strategy='most_frequent')), 
                ('onehotencoder',OneHotEncoder())
                ]

            )  

In [17]:
preprocessor=ColumnTransformer([
    ('num_pipeline',num_pipeline,numerical_cols),
    ('cat_pipeline',cat_pipeline,categorical_cols)

])

In [18]:
X_train=preprocessor.fit_transform(X_train)
X_test=preprocessor.transform(X_test)

In [19]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC

In [20]:
models={
    'Random Forest':RandomForestClassifier(),
    'Decision Tree':DecisionTreeClassifier(),
    'SVC':SVC()

}

In [21]:
from sklearn.metrics import accuracy_score

In [22]:
def evaluate_model(X_train,y_train,X_test,y_test,models):
    
    report = {}
    for i in range(len(models)):
        model = list(models.values())[i]
        # Train model
        model.fit(X_train,y_train)


        y_test_pred =model.predict(X_test)

        test_model_score = accuracy_score(y_test,y_test_pred)

        report[list(models.keys())[i]] =  test_model_score
            

            
    return report

In [23]:
evaluate_model(X_train,y_train,X_test,y_test,models)

{'Random Forest': 0.6530612244897959,
 'Decision Tree': 0.6122448979591837,
 'SVC': 0.6530612244897959}

In [24]:
classifier=RandomForestClassifier()

In [25]:
params={'max_depth':[3,5,10,None],
              'n_estimators':[100,200,300],
               'criterion':['gini','entropy']
              }

In [26]:
from sklearn.model_selection import RandomizedSearchCV

In [27]:
cv=RandomizedSearchCV(classifier,param_distributions=params,scoring='accuracy',cv=5,verbose=3)
cv.fit(X_train,y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV 1/5] END criterion=gini, max_depth=5, n_estimators=200;, score=0.590 total time=   0.3s
[CV 2/5] END criterion=gini, max_depth=5, n_estimators=200;, score=0.718 total time=   0.3s
[CV 3/5] END criterion=gini, max_depth=5, n_estimators=200;, score=0.615 total time=   0.4s
[CV 4/5] END criterion=gini, max_depth=5, n_estimators=200;, score=0.615 total time=   0.3s
[CV 5/5] END criterion=gini, max_depth=5, n_estimators=200;, score=0.615 total time=   0.3s
[CV 1/5] END criterion=gini, max_depth=10, n_estimators=100;, score=0.564 total time=   0.2s
[CV 2/5] END criterion=gini, max_depth=10, n_estimators=100;, score=0.641 total time=   0.2s
[CV 3/5] END criterion=gini, max_depth=10, n_estimators=100;, score=0.487 total time=   0.2s
[CV 4/5] END criterion=gini, max_depth=10, n_estimators=100;, score=0.564 total time=   0.2s
[CV 5/5] END criterion=gini, max_depth=10, n_estimators=100;, score=0.615 total time=   0.2s
[CV 1/5] END c

RandomizedSearchCV(cv=5, estimator=RandomForestClassifier(),
                   param_distributions={'criterion': ['gini', 'entropy'],
                                        'max_depth': [3, 5, 10, None],
                                        'n_estimators': [100, 200, 300]},
                   scoring='accuracy', verbose=3)

In [28]:
cv.best_params_

{'n_estimators': 100, 'max_depth': 5, 'criterion': 'entropy'}